# CH 7 MODEL EVALUATION

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

In [ ]:
import pandas as pd
import numpy as np 

In [ ]:
clothing_train = pd.read_csv("/content/gdrive/My Drive/Python Practice/Datasets/clothing_data_driven_training", delimiter='\t')
clothing_test = pd.read_csv("/content/gdrive/My Drive/Python Practice/Datasets/clothing_data_driven_test", delimiter='\t')

# HANDS-ON ANALYSIS

---
# Use **adult_ch6_training**, **adult_ch6_test** Dataset below.

---

# 23
Using the training data set, create a C5.0 model (Model 1) to predict a customer’s Income using Marital Status and Capital Gains and Losses. <br>
Obtain the predicted responses.





In [ ]:
import statsmodels.tools.tools as stattools
from sklearn.tree import DecisionTreeClassifier, export_graphviz

adult_tr = pd.read_csv("/content/gdrive/My Drive/Python Practice/Datasets/adult_ch6_training")
adult_test = pd.read_csv("/content/gdrive/My Drive/Python Practice/Datasets/adult_ch6_test")
y = adult_tr[['Income']]
y_test = adult_test[['Income']]

In [ ]:
mar_np = np.array(adult_tr['Marital status'])
(mar_cat, mar_cat_dict) = stattools.categorical(mar_np, drop=True, dictnames=True)

In [ ]:
mar_cat_pd = pd.DataFrame(mar_cat)
X = pd.concat((adult_tr[['Cap_Gains_Losses']], mar_cat_pd), axis=1)

X_names = ['Cap_Gains_Losses', 'Divorced', 'Married', 'Never-married', 'Separated', 'Widowed']
y_names = ['<=50K', '>50K']

c50_01 = DecisionTreeClassifier(criterion='entropy', max_leaf_nodes=5).fit(X,y)

In [ ]:
export_graphviz(c50_01, out_file = '/content/gdrive/My Drive/Python Practice/Datasets/c50_01.dot', 
                feature_names=X_names, class_names=y_names)

In [ ]:
c50_01.predict(X)

#24
Evaluate Model 1 using the test data set. <br>
Construct a contingency table to compare the actual and predicted values of Income.



In [ ]:
mar_np_test = np.array(adult_test['Marital status'])
(mar_cat_test, mar_cat_dict_test) = stattools.categorical(mar_np_test, drop=True, dictnames=True)

mar_cat_pd_test = pd.DataFrame(mar_cat_test)
X_test = pd.concat((adult_test[['Cap_Gains_Losses']], mar_cat_pd_test), axis=1)

In [ ]:
c50_01.predict(X_test)

In [ ]:
#y = adult_tr['Income']
y_test = adult_test['Income']

#pred_train = c50_01.predict(X)
pred_test = c50_01.predict(X_test)

In [ ]:
crosstab_model1 = pd.crosstab(pred_test, y_test, rownames=["prediction"], colnames=["actual"], margins=True)
crosstab_model1

#25
For Model 1, recapitulate Table 7.4 from the text, calculating all of the model evaluation measures shown in the table. <br>
Call this table the Model Evaluation Table. Leave space for Model 2.

In [ ]:
def eval(cross,lv):
    TN = cross[lv[0]][0]
    TP = cross[lv[1]][1]
    TAP = cross.sum(1)[1]
    TAN = cross.sum(1)[0]
    TPP = cross.sum(0)[1]
    GT = sum(cross.sum())

    acc = round((TN + TP)/GT, 2)    # Accuracy
    err = round(1-acc, 2)           # Error rate
    sen = round(TP/TAP, 2)          # Sensitivity
    rec = round(TN/TAN, 2)          # Specificity = Recall
    pre = round(TP/TPP, 2)          # Precision

    return[acc, err, sen, rec, pre]

def fscore(precision,recall, df):
    f = ((df**2+1)*precision*recall) / ((df**2)*precision + recall)
    return f     

acc1, err1, sen1, rec1, pre1 = eval(crosstab_model1, y_names)

In [ ]:
f1_model1 = fscore(pre1, rec1, 1)
f2_model1 = fscore(pre1, rec1, 2)
f05_model1 = fscore(pre1, rec1, 0.5)

In [ ]:
eval_dict = {'Eval Measures' : ['Accuracy', 'Error rate', 'Sensitivity', 'Recall(Specificity)', 'Precision', 'F1', 'F2', 'F0.5'],
             'Model 1 : Eval Values' : [acc1, err1, sen1, rec1, pre1, f1_model1, f2_model1, f05_model1]} 

model_eval = pd.DataFrame(eval_dict)
model_eval

#26
Clearly and completely interpret each of the Model 1 evaluation measures from the Model Evaluation Table.

#27
Create a cost matrix, called the 3x cost matrix, that specifies a false positive is four times as bad as a false negative.


In [ ]:
#cost_mat = np.array(([0,4],[1,0]))
#cost_mat

In [ ]:
n = adult_tr.shape[0]
#cost_list = [0,4,1,0]*n
cost_list = [4,1,0,0]*n
cost_mat = np.array(cost_list).reshape(n,4)
cost_mat

#28
Using the training data set, build a C5.0 model (Model 2) to predict a customer’s Income using Marital Status and Capital Gains and Losses, using the 3x cost matrix.


In [ ]:
#!pip install costcla

In [ ]:
 import costcla.models as co

# change y to array and change values to numeric (<=50K = 0)
y_train = np.array(y).reshape(n,)
y_train[y_train == y_names[0]] = 0
y_train[y_train == y_names[1]] = 1

# change X, X_test (training, test set) to array
X_array = np.array(X)
X_test_array = np.array(X_test)

# Create C5.0 model (Model 2) using Cost Matrix
# Fit training set to this model, and Obtain prediction
DT = co.CostSensitiveDecisionTreeClassifier()
DT_fit = DT.fit(X_array, y_train, cost_mat=cost_mat)
DT_fit.predict(X_array)

#29
Evaluate your predictions from Model 2 using the actual response values from the test data set. <br>
Add Overall Model Cost and Profit per Customer to the Model Evaluation Table.<br>
Calculate all the measures from the Model Evaluation Table.


In [ ]:
pred_test = DT_fit.predict(X_test_array)
pred_test = pred_test.reshape(X_test_array.shape[0], 1)

y_pred_test = pd.DataFrame(pred_test)
y_pred_test[y_pred_test == 0] = y_names[0]
y_pred_test[y_pred_test == 1] = y_names[1]


crosstab_model2 = pd.crosstab(adult_test['Income'], y_pred_test.loc[:,0], rownames=['Actual'], colnames=['Predicted'])
crosstab_model2

In [ ]:
def eval(cross,lv):
    TN = cross[lv[0]][0]
    TP = cross[lv[1]][1]
    TAP = cross.sum(1)[1]
    TAN = cross.sum(1)[0]
    TPP = cross.sum(0)[1]
    GT = sum(cross.sum())

    acc = round((TN + TP)/GT, 2)    # Accuracy
    err = round(1-acc, 2)           # Error rate
    sen = round(TP/TAP, 2)          # Sensitivity
    rec = round(TN/TAN, 2)          # Specificity = Recall
    pre = round(TP/TPP, 2)          # Precision

    return[acc, err, sen, rec, pre]

def fscore(precision,recall, df):
    f = ((df**2+1)*precision*recall) / ((df**2)*precision + recall)
    return f     

acc2, err2, sen2, rec2, pre2 = eval(crosstab_model2, y_names)

In [ ]:
f1_model2 = fscore(pre2, rec2, 1)
f2_model2 = fscore(pre2, rec2, 2)
f05_model2 = fscore(pre2, rec2, 0.5)

In [ ]:
eval_model2 = {'Model 2 : Eval Values' :  [acc2, err2, sen2, rec2, pre2, f1_model2, f2_model2, f05_model2]} 
eval_model2_summary = pd.DataFrame(eval_model2)

model_eval = pd.concat((model_eval, eval_model2_summary), axis=1)
model_eval

In [ ]:
def eval(cross,lv,cost_mat):
    TN = cross[lv[0]][0]
    FN = cross[lv[1]][0]
    FP = cross[lv[0]][1]
    TP = cross[lv[1]][1]
    GT = sum(cross.sum())

    total_cost = FP*cost_mat[0] + FN*cost_mat[1] + TP*cost_mat[2] + TN*cost_mat[3]  
    profit_per_customer = -total_cost / GT
    return[total_cost, profit_per_customer]

In [ ]:
# Overall Model Cost / Profit per Customer for MODEL 2
cost_model2, profit_model2 = eval(crosstab_model2, y_names, cost_mat[1])
print('overall model cost: %.2f & Profit per Customer: %.3f' % (cost_model2, profit_model2))

#30
Compare the evaluation measures from Model 1 and Model 2 using the 3x cost matrix. <br>
Discuss the strengths and weaknesses of each model.